In [1]:
import os
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import requests
from datetime import datetime as dt

In [2]:
ALPHAVANTAGE_KEY = os.getenv('ALPHAVANTAGE_KEY')
assert ALPHAVANTAGE_KEY is not None, "You must export ALPHAVANTAGE_KEY..."
ALPHAVANTAGE_KEY

'032HZXCB7T21N7SC'

In [3]:
SAMPLE_START_DATE = dt.fromisoformat('2019-12-19').date()
SAMPLE_END_DATE = dt.fromisoformat('2023-08-31').date()
(SAMPLE_START_DATE, SAMPLE_END_DATE)

(datetime.date(2019, 12, 19), datetime.date(2023, 8, 31))

In [4]:
DIGITAL_CURRENCY_URL = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey={}&datatype=csv'.format(ALPHAVANTAGE_KEY)
SPY_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={apikey}&datatype=csv'.format(symbol='SPY', apikey=ALPHAVANTAGE_KEY)
QQQ_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={apikey}&datatype=csv'.format(symbol='QQQ', apikey=ALPHAVANTAGE_KEY)
(DIGITAL_CURRENCY_URL, SPY_URL, QQQ_URL)

('https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=USD&apikey=032HZXCB7T21N7SC&datatype=csv',
 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=SPY&outputsize=full&apikey=032HZXCB7T21N7SC&datatype=csv',
 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=QQQ&outputsize=full&apikey=032HZXCB7T21N7SC&datatype=csv')

In [5]:
%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format
sns.set_style("whitegrid")

mpl.rcParams.update({
                     'text.color' : "white",
                     'lines.linewidth': 3,
                     'font.size': 16.0,
                     'legend.facecolor': "182742",
                     'legend.edgecolor': 'CFB023',
                     'legend.fancybox': True,
                     'legend.framealpha': 1.,
                     'legend.loc': 'right',
                     'xtick.color': 'white',
                     'ytick.color': 'white',
                     'ytick.minor.visible': True,
                     'axes.facecolor': '#182742',
                     'axes.edgecolor': 'white',
#                      'axes.grid.axis': 'y',
#                      'axes.grid.which': 'minor',
                     'axes.labelcolor' : "white",
                     'axes.labelsize': "24",
                     'axes.labelweight': "bold",
                     'axes.titlesize': 28,
                     'axes.titleweight': 'bold',
                     'axes.titlepad': 20,
                     'figure.facecolor': '#182742',
#                      'xtick.bottom': False,
                    }
                   )

In [6]:
def total_return(prices):
  return prices.iloc[-1] / prices.iloc[0] - 1

# Download Bitcoin Historical Data

In [7]:
# TODO: Figure out how to add error checking
# r = requests.get(URL)
# data = r.json()
# assert 'Error Message' not in data.keys(), "Error Received. {}".format(data['Error Message'])

df = pd.read_csv(DIGITAL_CURRENCY_URL)
df['timestamp'] = pd.to_datetime(df.timestamp)
df.index = df.timestamp.dt.date
df = df[::-1]
df.rename(columns={"close (USD)": "btc_close"}, inplace=True)
df = df.drop(['open (USD)', 'high (USD)', 'low (USD)', 'open (USD).1', 'high (USD).1', 'low (USD).1', 'close (USD).1'], axis=1)
df['pct_change'] = df['btc_close'].pct_change()
df[['timestamp', 'btc_close', 'pct_change']].to_csv('bitcoin_daily_returns.csv')
df[['timestamp', 'btc_close', 'pct_change']]
df

,timestamp,btc_close,volume,market cap (USD),pct_change
timestamp,,,,,
2020-12-11,2020-12-11,"18,036.5300","72,610.7243","72,610.7243",NaN
2020-12-12,2020-12-12,"18,808.6900","49,519.9784","49,519.9784",0.0428
2020-12-13,2020-12-13,"19,174.9900","56,560.8217","56,560.8217",0.0195
2020-12-14,2020-12-14,"19,273.1400","47,257.2013","47,257.2013",0.0051
2020-12-15,2020-12-15,"19,426.4300","61,834.3660","61,834.3660",0.0080
...,...,...,...,...,...
2023-09-02,2023-09-02,"25,869.5100","16,250.7770","16,250.7770",0.0025
2023-09-03,2023-09-03,"25,971.2100","17,474.4767","17,474.4767",0.0039
2023-09-04,2023-09-04,"25,826.0200","21,777.5961","21,777.5961",-0.0056


In [8]:
btc_monthly_returns = df.groupby([df.timestamp.dt.year, df.timestamp.dt.month])['btc_close'].apply(total_return)
btc_monthly_returns.to_csv("bitcoin_monthly_returns.csv")
btc_monthly_returns

timestamp  timestamp
2020       12           0.6036
2021       1            0.1282
           2            0.3463
           3            0.1846
           4           -0.0175
           5           -0.3555
           6           -0.0449
           7            0.2375
           8            0.1821
           9           -0.1022
           10           0.2733
           11          -0.0650
           12          -0.1918
2022       1           -0.1939
           2            0.1154
           3            0.0245
           4           -0.1869
           5           -0.1733
           6           -0.3309
           7            0.2082
           8           -0.1383
           9           -0.0352
           10           0.0611
           11          -0.1621
           12          -0.0256
2023       1            0.3917
           2           -0.0249
           3            0.2047
           4            0.0274
           5           -0.0306
           6            0.1363
           7      

In [9]:

def read_equities(url, start_date, end_date, outfile):
    df = pd.read_csv(url)
    df['timestamp'] = pd.to_datetime(df.timestamp)
    df.index = df.timestamp.dt.date
    df = df[::-1]
    sample = df[start_date:end_date]
    # df.rename(columns={"close (USD)": "btc_close"}, inplace=True)
    # df = df.drop(['open (USD)', 'high (USD)', 'low (USD)', 'open (USD).1', 'high (USD).1', 'low (USD).1', 'close (USD).1'], axis=1)
    sample['pct_change'] = sample['close'].pct_change().copy()
    sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
    sample['growth_of_10k'].iloc[0] = 10000.
#     sample.insert(4, 'growth_of_10k', 0.)
#     sample['growth_of_10k'].iloc[0] = 10000.0
# #     print(sample.iloc[1:]['growth_of_10k'])
#     print(sample['growth_of_10k'].shift(1))
#     print((1 + sample['pct_change']))
#     sample['growth_of_10k'] = sample['growth_of_10k'].shift(1) * (1 + sample['pct_change'])
    sample[['timestamp', 'close', 'pct_change', 'growth_of_10k']].to_csv(outfile)
    return sample[['timestamp', 'close', 'pct_change', 'growth_of_10k']]

In [10]:
spy_history = read_equities(SPY_URL, SAMPLE_START_DATE, SAMPLE_END_DATE, 'spy_history.csv')
spy_history

/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_35658/43137134.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['pct_change'] = sample['close'].pct_change().copy()
/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_35658/43137134.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_35658/43137134.py:11: SettingWithCopyWarning: 
A value is t

,timestamp,close,pct_change,growth_of_10k
timestamp,,,,
2019-12-19,2019-12-19,320.9000,NaN,"10,000.0000"
2019-12-20,2019-12-20,320.7300,-0.0005,"9,994.7024"
2019-12-23,2019-12-23,321.2200,0.0015,"10,009.9720"
2019-12-24,2019-12-24,321.2300,0.0000,"10,010.2836"
2019-12-26,2019-12-26,322.9400,0.0053,"10,063.5712"
...,...,...,...,...
2023-08-25,2023-08-25,439.9700,0.0070,"13,710.5017"
2023-08-28,2023-08-28,442.7600,0.0063,"13,797.4447"
2023-08-29,2023-08-29,449.1600,0.0145,"13,996.8838"


In [11]:
qqq_history = read_equities(QQQ_URL, SAMPLE_START_DATE, SAMPLE_END_DATE, 'qqq_history.csv')
qqq_history

/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_35658/43137134.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['pct_change'] = sample['close'].pct_change().copy()
/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_35658/43137134.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['growth_of_10k'] = (sample['pct_change'].iloc[1:] + 1.).cumprod() * 10000.
/var/folders/22/_p71mrgd08g3swgccq1wnd1m0000gp/T/ipykernel_35658/43137134.py:11: SettingWithCopyWarning: 
A value is t

,timestamp,close,pct_change,growth_of_10k
timestamp,,,,
2019-12-19,2019-12-19,210.8600,NaN,"10,000.0000"
2019-12-20,2019-12-20,211.7100,0.0040,"10,040.3111"
2019-12-23,2019-12-23,211.8100,0.0005,"10,045.0536"
2019-12-24,2019-12-24,211.9200,0.0005,"10,050.2703"
2019-12-26,2019-12-26,213.7900,0.0088,"10,138.9548"
...,...,...,...,...
2023-08-25,2023-08-25,364.0200,0.0078,"17,263.5872"
2023-08-28,2023-08-28,366.7600,0.0075,"17,393.5313"
2023-08-29,2023-08-29,374.7700,0.0218,"17,773.4042"
